# Train model for horizon



In [ ]:
!pip install hopsworks==4.2.*

  Using cached hopsworks-4.2.9-py3-none-any.whl.metadata (11 kB)
  Using cached pyhumps-1.6.1-py3-none-any.whl.metadata (3.7 kB)
  Using cached furl-2.1.4-py2.py3-none-any.whl.metadata (25 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 665.0/665.0 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.6/258.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 99.2 MB/s eta 0

In [ ]:
import os
import joblib
import pandas as pd
import numpy as np
import hopsworks
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# -------------------------------
# 1. CONNECT TO HOPSWORKS
# -------------------------------
project = hopsworks.login()
fs = project.get_feature_store()
mr = project.get_model_registry()
print("✅ Connected to project:", project.name)

# -------------------------------
# 2. LOAD FEATURE GROUP
# -------------------------------
fg = fs.get_feature_group("model_features", version=1)
df = fg.read()
print("✅ Loaded feature data. Shape:", df.shape)

# -------------------------------
# 3. DEFINE AQI CONVERSION
# -------------------------------
# These are approximate midpoint values for each AQI category (1–5)
aqi_scale_map = {
    1: 50,    # Good (0–50)
    2: 100,   # Moderate (51–100)
    3: 150,   # Unhealthy for Sensitive Groups (101–150)
    4: 200,   # Unhealthy (151–200)
    5: 300,   # Very Unhealthy (201–300+)
}

# Convert all columns containing 'aqi' with scale 1–5 → actual numeric AQI
aqi_cols = [c for c in df.columns if 'aqi' in c.lower()]
for col in aqi_cols:
    df[col] = df[col].map(aqi_scale_map).fillna(df[col])

print(f"🔄 Converted scaled AQI columns to numeric AQI for {len(aqi_cols)} columns")

# -------------------------------
# 4. PREPARE DATA
# -------------------------------
H = 72  # forecast horizon in hours
target_col = f"aqi_t_plus_{H}"

if target_col not in df.columns:
    raise ValueError(f"❌ Target column '{target_col}' not found in feature group.")

# Drop only where target is missing
df_sup = df.dropna(subset=[target_col]).copy()

# Columns not to use as features
non_feature_cols = ["datetime", "timestamp"]
features = [c for c in df_sup.columns if c not in non_feature_cols + [target_col]]

print(f"📊 Total features before cleaning: {len(features)}")

# Handle missing / invalid input values safely
X = df_sup[features].copy()

# Replace invalid pollutant readings (e.g., negatives or zeros)
X = X.replace([np.inf, -np.inf], np.nan)
X[X <= 0] = np.nan

# Forward fill only (avoid leaking future data)
X = X.ffill()

# Drop rows still having missing values after ffill
missing_before = X.isna().sum().sum()
X = X.dropna()
missing_after = X.isna().sum().sum()

print(f"🧹 Cleaned missing values: {missing_before} → {missing_after}")

# Align target with cleaned feature set
y = df_sup.loc[X.index, target_col].astype(float)

# Time-aware split (no shuffle)
split_frac = 0.8
split_idx = int(len(X) * split_frac)
X_train, X_val = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_val = y.iloc[:split_idx], y.iloc[split_idx:]
print(f"✅ Data prepared | Train: {len(X_train)}, Val: {len(X_val)}")

# -------------------------------
# 5. TRAIN & EVALUATE MODELS
# -------------------------------
def metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

models = {
    "RandomForest": RandomForestRegressor(
        n_estimators=200, random_state=42, n_jobs=-1
    ),
    "Ridge": Ridge(alpha=1.0, random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42)
}

results = []

for name, model in models.items():
    print(f"\n🚀 Training {name} ...")
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    mae, rmse, r2 = metrics(y_val, preds)
    results.append({"Model": name, "MAE": mae, "RMSE": rmse, "R2": r2})
    print(f"{name} -> MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.3f}")

results_df = pd.DataFrame(results).sort_values(by="RMSE")
print("\n📊 Model Comparison:\n", results_df)

# -------------------------------
# 6. REGISTER BEST MODEL
# -------------------------------
best_model_name = results_df.iloc[0]["Model"]
best_model = models[best_model_name]
best_metrics = results_df.iloc[0]

print(f"\n🏆 Best Model: {best_model_name}")
print(f"📈 Metrics -> MAE: {best_metrics['MAE']:.2f}, RMSE: {best_metrics['RMSE']:.2f}, R²: {best_metrics['R2']:.3f}")

# Save best model locally
model_dir = "models"
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, f"{best_model_name}_H{H}.pkl")
joblib.dump(best_model, model_path)
print(f"💾 Model saved locally at: {model_path}")

# Create schema
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

# Register model
model = mr.sklearn.create_model(
    name=f"AQI_{best_model_name}_H{H}",
    metrics={
        "mae": float(best_metrics["MAE"]),
        "rmse": float(best_metrics["RMSE"]),
        "r2": float(best_metrics["R2"]),
    },
    model_schema=model_schema,
    description=f"{best_model_name} model trained for exact AQI prediction ({H}-hour horizon)"
)
model.save(model_path)

print(f"\n✅ Registered model '{best_model_name}' in Hopsworks (version {model.version})")


Connection closed.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1251499
✅ Connected to project: pearls_aqi_predictor
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.98s) 
✅ Loaded feature data. Shape: (6776, 155)
🔄 Converted scaled AQI columns to numeric AQI for 19 columns
📊 Total features before cleaning: 153
🧹 Cleaned missing values: 6 → 0
✅ Data prepared | Train: 5360, Val: 1341

🚀 Training RandomForest ...
RandomForest -> MAE: 19.64, RMSE: 36.11, R²: 0.748

🚀 Training Ridge ...
Ridge -> MAE: 39.87, RMSE: 55.88, R²: 0.397

🚀 Training GradientBoosting ...
GradientBoosting -> MAE: 30.83, RMSE: 46.31, R²: 0.586

📊 Model Comparison:
               Model        MAE       RMSE        R2
0      RandomForest  19.643446  36.113914  0.748284
2  GradientBoosting  30.832045  46.314794  0.585999
1             Ridge  39.867615  55.882982  0.397272

🏆 Best Model: RandomForest
📈 Metrics -> MAE: 19.64, RMSE: 36.11, R²: 0.748
💾 Model saved lo

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading /content/models/RandomForest_H72.pkl: 0.000%|          | 0/10913249 elapsed<00:00 remaining<?

Uploading /content/model_schema.json: 0.000%|          | 0/11837 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1251499/models/AQI_RandomForest_H72/1

✅ Registered model 'RandomForest' in Hopsworks (version 1)
